In [5]:
import sqlite3
import pandas as pd
import json
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

In [6]:
movies_df = pd.read_csv('../data/raw/tmdb_5000_movies.csv')
movies_df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [7]:
credits_df = pd.read_csv('../data/raw/tmdb_5000_credits.csv')
credits_df.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


<h3>Unificar dataframes en una base de datos</h3>

In [8]:
conn = sqlite3.connect('movies_database.db')

movies_df.to_sql('movies', conn, if_exists='replace', index=False)
credits_df.to_sql('credits', conn, if_exists='replace', index=False)

try:
    query = """CREATE TABLE unified_movies AS SELECT m.*, c.* FROM movies m JOIN credits c ON m.title = c.title;"""
    cursor = conn.cursor()
    cursor.execute(query)
    conn.commit()
except sqlite3.OperationalError as e:
    if "already exists" in str(e):
        print("Table already exists")
    else:
        raise

unified_df = pd.read_sql('SELECT * FROM unified_movies', conn)

conn.close()

<h3>Limpiar columnas innecesarias</h3>

In [9]:
conn = sqlite3.connect('movies_database.db')

cursor = conn.cursor()

cursor.execute("DROP TABLE IF EXISTS cleaned_unified_movies;")
conn.commit()

try:
    query = """CREATE TABLE cleaned_unified_movies AS SELECT "movie_id", "title", "overview", "genres", "keywords", "cast", "crew"FROM unified_movies;"""
    cursor.execute(query)
    conn.commit()
except sqlite3.OperationalError as e:
    if "already exists" in str(e):
        print("Table already exists")
    else:
        raise

cleaned_unified_df = pd.read_sql('SELECT * FROM cleaned_unified_movies', conn)

conn.close()

<h3>Transformar datos</h3>

In [10]:
conn = sqlite3.connect('movies_database.db')
cleaned_unified_df = pd.read_sql('SELECT * FROM cleaned_unified_movies', conn)

def extract_names(json_str):
    try:
        data = json.loads(json_str)
        return [item['name'] for item in data]
    except (json.JSONDecodeError, TypeError):
        return []

cleaned_unified_df['genres'] = cleaned_unified_df['genres'].apply(extract_names)
cleaned_unified_df['keywords'] = cleaned_unified_df['keywords'].apply(extract_names)

def extract_top_cast(json_str, top_n=3):
    try:
        data = json.loads(json_str)
        return [item['name'] for item in data[:top_n]]
    except (json.JSONDecodeError, TypeError):
        return []

cleaned_unified_df['cast'] = cleaned_unified_df['cast'].apply(lambda x: extract_top_cast(x, top_n=3))

cleaned_unified_df['genres'] = cleaned_unified_df['genres'].apply(lambda x: ', '.join(x) if isinstance(x, list) else '')
cleaned_unified_df['keywords'] = cleaned_unified_df['keywords'].apply(lambda x: ', '.join(x) if isinstance(x, list) else '')
cleaned_unified_df['cast'] = cleaned_unified_df['cast'].apply(lambda x: ', '.join(x) if isinstance(x, list) else '')

cleaned_unified_df.to_sql('final_cleaned_movies', conn, if_exists='replace', index=False)

conn.close()

In [11]:
def extract_director(json_str):
    try:
        data = json.loads(json_str)
        for person in data:
            if person.get('job') == 'Director':
                return person.get('name', '')
        return ''
    except (json.JSONDecodeError, TypeError):
        return ''

def convert_overview_to_list(text):
    if isinstance(text, str):
        return text.split()
    return []

def convert_list_to_string(lst):
    if isinstance(lst, list):
        return json.dumps(lst)
    return lst

conn = sqlite3.connect('movies_database.db')

cleaned_unified_df['crew'] = cleaned_unified_df['crew'].apply(extract_director)
cleaned_unified_df['overview'] = cleaned_unified_df['overview'].apply(convert_overview_to_list)
cleaned_unified_df['overview'] = cleaned_unified_df['overview'].apply(convert_list_to_string)
cleaned_unified_df.to_sql('final_cleaned_movies', conn, if_exists='replace', index=False)

conn.close()


In [12]:
cleaned_unified_df.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[""In"", ""the"", ""22nd"", ""century,"", ""a"", ""parapl...","Action, Adventure, Fantasy, Science Fiction","culture clash, future, space war, space colony...","Sam Worthington, Zoe Saldana, Sigourney Weaver",James Cameron
1,285,Pirates of the Caribbean: At World's End,"[""Captain"", ""Barbossa,"", ""long"", ""believed"", ""...","Adventure, Fantasy, Action","ocean, drug abuse, exotic island, east india t...","Johnny Depp, Orlando Bloom, Keira Knightley",Gore Verbinski
2,206647,Spectre,"[""A"", ""cryptic"", ""message"", ""from"", ""Bond\u201...","Action, Adventure, Crime","spy, based on novel, secret agent, sequel, mi6...","Daniel Craig, Christoph Waltz, Léa Seydoux",Sam Mendes
3,49026,The Dark Knight Rises,"[""Following"", ""the"", ""death"", ""of"", ""District""...","Action, Crime, Drama, Thriller","dc comics, crime fighter, terrorist, secret id...","Christian Bale, Michael Caine, Gary Oldman",Christopher Nolan
4,49529,John Carter,"[""John"", ""Carter"", ""is"", ""a"", ""war-weary,"", ""f...","Action, Adventure, Science Fiction","based on novel, mars, medallion, space travel,...","Taylor Kitsch, Lynn Collins, Samantha Morton",Andrew Stanton


In [13]:
def remove_spaces(text):
    """Elimina los espacios de una cadena de texto."""
    if isinstance(text, str):
        return text.replace(" ", "")
    return text

conn = sqlite3.connect('movies_database.db')

columns_to_clean = ['genres', 'cast', 'crew', 'keywords']
for col in columns_to_clean:
    cleaned_unified_df[col] = cleaned_unified_df[col].apply(remove_spaces)

cleaned_unified_df.to_sql('final_cleaned_movies', conn, if_exists='replace', index=False)

conn.close()

In [14]:
cleaned_unified_df.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[""In"", ""the"", ""22nd"", ""century,"", ""a"", ""parapl...","Action,Adventure,Fantasy,ScienceFiction","cultureclash,future,spacewar,spacecolony,socie...","SamWorthington,ZoeSaldana,SigourneyWeaver",JamesCameron
1,285,Pirates of the Caribbean: At World's End,"[""Captain"", ""Barbossa,"", ""long"", ""believed"", ""...","Adventure,Fantasy,Action","ocean,drugabuse,exoticisland,eastindiatradingc...","JohnnyDepp,OrlandoBloom,KeiraKnightley",GoreVerbinski
2,206647,Spectre,"[""A"", ""cryptic"", ""message"", ""from"", ""Bond\u201...","Action,Adventure,Crime","spy,basedonnovel,secretagent,sequel,mi6,britis...","DanielCraig,ChristophWaltz,LéaSeydoux",SamMendes
3,49026,The Dark Knight Rises,"[""Following"", ""the"", ""death"", ""of"", ""District""...","Action,Crime,Drama,Thriller","dccomics,crimefighter,terrorist,secretidentity...","ChristianBale,MichaelCaine,GaryOldman",ChristopherNolan
4,49529,John Carter,"[""John"", ""Carter"", ""is"", ""a"", ""war-weary,"", ""f...","Action,Adventure,ScienceFiction","basedonnovel,mars,medallion,spacetravel,prince...","TaylorKitsch,LynnCollins,SamanthaMorton",AndrewStanton


In [15]:
def combine_columns_into_tags(df):
    df['tags'] = df['overview'] + ' ' + df['genres'] + ' ' + df['cast'] + ' ' + df['crew'] + ' ' + df['keywords']
    df['tags'] = df['tags'].str.replace(',', ' ', regex=False)
    df['tags'] = df['tags'].str.strip()
    
    return df

conn = sqlite3.connect('movies_database.db')
cleaned_unified_df = combine_columns_into_tags(cleaned_unified_df)
cleaned_unified_df.to_sql('final_cleaned_movies', conn, if_exists='replace', index=False)

conn.close()

In [16]:
cleaned_unified_df.head()

,movie_id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[""In"", ""the"", ""22nd"", ""century,"", ""a"", ""parapl...","Action,Adventure,Fantasy,ScienceFiction","cultureclash,future,spacewar,spacecolony,socie...","SamWorthington,ZoeSaldana,SigourneyWeaver",JamesCameron,"[""In"" ""the"" ""22nd"" ""century "" ""a"" ""parapl..."
1,285,Pirates of the Caribbean: At World's End,"[""Captain"", ""Barbossa,"", ""long"", ""believed"", ""...","Adventure,Fantasy,Action","ocean,drugabuse,exoticisland,eastindiatradingc...","JohnnyDepp,OrlandoBloom,KeiraKnightley",GoreVerbinski,"[""Captain"" ""Barbossa "" ""long"" ""believed"" ""..."
2,206647,Spectre,"[""A"", ""cryptic"", ""message"", ""from"", ""Bond\u201...","Action,Adventure,Crime","spy,basedonnovel,secretagent,sequel,mi6,britis...","DanielCraig,ChristophWaltz,LéaSeydoux",SamMendes,"[""A"" ""cryptic"" ""message"" ""from"" ""Bond\u201..."
3,49026,The Dark Knight Rises,"[""Following"", ""the"", ""death"", ""of"", ""District""...","Action,Crime,Drama,Thriller","dccomics,crimefighter,terrorist,secretidentity...","ChristianBale,MichaelCaine,GaryOldman",ChristopherNolan,"[""Following"" ""the"" ""death"" ""of"" ""District""..."
4,49529,John Carter,"[""John"", ""Carter"", ""is"", ""a"", ""war-weary,"", ""f...","Action,Adventure,ScienceFiction","basedonnovel,mars,medallion,spacetravel,prince...","TaylorKitsch,LynnCollins,SamanthaMorton",AndrewStanton,"[""John"" ""Carter"" ""is"" ""a"" ""war-weary "" ""f..."


<h3>Vectorizar texto</h3>

In [17]:
def clean_text(df, column):
    df[column] = (
        df[column]
        .str.strip()
        .str.lower()
        .str.replace(r"[^\w\s]", "", regex=True)
    )

    return df

text_columns = ['title', 'overview', 'genres', 'keywords', 'cast', 'crew', 'tags']

for column in text_columns:
    clean_text(cleaned_unified_df, column)

cleaned_unified_df.head()

,movie_id,title,overview,genres,keywords,cast,crew,tags
0,19995,avatar,in the 22nd century a paraplegic marine is dis...,actionadventurefantasysciencefiction,cultureclashfuturespacewarspacecolonysocietysp...,samworthingtonzoesaldanasigourneyweaver,jamescameron,in the 22nd century a paraplegic marine...
1,285,pirates of the caribbean at worlds end,captain barbossa long believed to be dead has ...,adventurefantasyaction,oceandrugabuseexoticislandeastindiatradingcomp...,johnnydepporlandobloomkeiraknightley,goreverbinski,captain barbossa long believed to be de...
2,206647,spectre,a cryptic message from bondu2019s past sends h...,actionadventurecrime,spybasedonnovelsecretagentsequelmi6britishsecr...,danielcraigchristophwaltzléaseydoux,sammendes,a cryptic message from bondu2019s past s...
3,49026,the dark knight rises,following the death of district attorney harve...,actioncrimedramathriller,dccomicscrimefighterterroristsecretidentitybur...,christianbalemichaelcainegaryoldman,christophernolan,following the death of district attorney ...
4,49529,john carter,john carter is a warweary former military capt...,actionadventuresciencefiction,basedonnovelmarsmedallionspacetravelprincessal...,taylorkitschlynncollinssamanthamorton,andrewstanton,john carter is a warweary former milita...


In [18]:
vec_model = CountVectorizer(stop_words="english", max_features=1000)

vectorized_columns = []

for column in text_columns:
    if cleaned_unified_df[column].dtype == 'object':
        X = vec_model.fit_transform(cleaned_unified_df[column].dropna())

        vectorized_array = X.toarray()

        vectorized_df = pd.DataFrame(vectorized_array, columns=vec_model.get_feature_names_out())

        vectorized_columns.append(vectorized_df)

if vectorized_columns:
    vectorized_df = pd.concat(vectorized_columns, axis=1)
    cleaned_unified_df = pd.concat([cleaned_unified_df, vectorized_df], axis=1)

cleaned_unified_df.head()

,movie_id,title,overview,genres,keywords,cast,crew,tags,10,12,...,worst,writer,wrong,year,years,york,young,younger,youth,zombie
0,19995,avatar,in the 22nd century a paraplegic marine is dis...,actionadventurefantasysciencefiction,cultureclashfuturespacewarspacecolonysocietysp...,samworthingtonzoesaldanasigourneyweaver,jamescameron,in the 22nd century a paraplegic marine...,0,0,...,0,0,0,0,0,0,0,0,0,0
1,285,pirates of the caribbean at worlds end,captain barbossa long believed to be dead has ...,adventurefantasyaction,oceandrugabuseexoticislandeastindiatradingcomp...,johnnydepporlandobloomkeiraknightley,goreverbinski,captain barbossa long believed to be de...,0,0,...,0,0,0,0,0,0,0,0,0,0
2,206647,spectre,a cryptic message from bondu2019s past sends h...,actionadventurecrime,spybasedonnovelsecretagentsequelmi6britishsecr...,danielcraigchristophwaltzléaseydoux,sammendes,a cryptic message from bondu2019s past s...,0,0,...,0,0,0,0,0,0,0,0,0,0
3,49026,the dark knight rises,following the death of district attorney harve...,actioncrimedramathriller,dccomicscrimefighterterroristsecretidentitybur...,christianbalemichaelcainegaryoldman,christophernolan,following the death of district attorney ...,0,0,...,0,0,0,0,1,0,0,0,0,0
4,49529,john carter,john carter is a warweary former military capt...,actionadventuresciencefiction,basedonnovelmarsmedallionspacetravelprincessal...,taylorkitschlynncollinssamanthamorton,andrewstanton,john carter is a warweary former milita...,0,0,...,0,0,0,0,0,0,0,0,0,0


<h4>Construit KNN</h4>

In [19]:
X = vec_model.fit_transform(cleaned_unified_df[column].dropna())

vectors = X.toarray()
similarity = cosine_similarity(vectors)

knn = NearestNeighbors(n_neighbors=5, metric='cosine')
knn.fit(vectors)

distances, indices = knn.kneighbors([vectors[0]])

print("Índices:", indices)
print("Distancias:", distances)


Índices: [[   0  507 1192 1916 1214]]
Distancias: [[0.         0.47076225 0.54239568 0.55767413 0.56356422]]


In [20]:
def recommend(movie):
    movie_index = cleaned_unified_df[cleaned_unified_df["title"] == movie]
    
    if movie_index.empty:
        print(f"La película '{movie}' no se encuentra en la base de datos.")
        return
    
    movie_index = movie_index.index[0]
    distances = similarity[movie_index]
    
    movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    
    for i in movie_list:
        print(cleaned_unified_df.iloc[i[0]].title)


In [23]:
recommend('the crow')

oldboy
dragonball evolution
superman returns
sucker punch
a man apart
